In [1]:
env PYTHONHASHSEED=0

env: PYTHONHASHSEED=0


In [2]:
#import relevant libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pygam import LinearGAM, GAM, s, l, te
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import os
import csv
import shutil
import traceback
import random
from datetime import datetime

import geopandas as gpd
import fiona
import shapely
from shapely import LineString
from shapely.geometry import Point
from shapely.ops import unary_union

In [3]:
def import_csv(input_no, *csvs):
    if input_no < 1 or input_no > len(csvs):
        raise ValueError("input_no should be between 1 and the number of CSV files")
    
    dfs = [pd.read_csv(csv, dtype={"ptcode": str}) for csv in csvs[:input_no]]
    full_wims = pd.concat(dfs)
    
    return full_wims#def import class


In [4]:
def create_pivot(df):
    #Create Pivot
    Est_pivot = df.pivot_table(values='result',index=['date','time','ptcode','easting','northing','site_avg_salinity','salinity_class'],columns=['detname'])
    #Drop the "Time" column from results as this duplicates a column elsewhere
    Est_pivot = Est_pivot.drop(columns = ['time'],axis=1,errors='ignore')
    #Reset the index
    Est_pivot.reset_index(inplace=True)
    #Create a column for year
    Est_pivot['date'] = pd.to_datetime(Est_pivot['date'])
    Est_pivot["year"]=Est_pivot["date"].dt.year
    #Create a column for month
    Est_pivot["month"]=Est_pivot["date"].dt.month
    Est_pivot.columns = Est_pivot.columns.str.replace(':', '')
    Est_pivot.columns = Est_pivot.columns.str.replace('%', '')
    Est_group_all = Est_pivot.copy().reset_index()

    return Est_group_all

In [5]:
def date_to_float_year(date):
    # Convert the timestamp to a string in the format '%Y-%m-%d'
    date_str = date.strftime('%Y-%m-%d')
    
    # Convert the date string to a datetime object
    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
    
    # Extract year, month, and day from the date object
    year = date_obj.year
    month = date_obj.month
    day = date_obj.day
    
    # Calculate the fractional part of the year
    fractional_year = year + (month - 1) / 12 + (day - 1) / 365
    
    return fractional_year

In [6]:
def filter_by_det(Est_group_all,det, minimum_day_range = 3653, minimum_sample_count = 120):

    # Group by 'ptcode' and find the earliest and latest date
    date_ranges = Est_group_all.groupby('ptcode')['date'].agg(['min', 'max']).reset_index()
    # Calculate the difference in days between the earliest and latest dates
    date_ranges['day_range'] = (date_ranges['max'] - date_ranges['min']).dt.days
    # Filter date ranges where the difference is less than 3653 days (10 years)
    date_ranges_filtered = date_ranges[date_ranges['day_range'] > minimum_day_range]
    
    # Bring together the data only where ptcodes exist in both dataframes
    merged_df = pd.merge(Est_group_all, date_ranges_filtered[['ptcode']], on='ptcode', how='inner')

    # Group by pintcode and produce states
    Est_group = merged_df.groupby(['ptcode'])[det].agg(['mean','std','count']).reset_index()
    # Remove any samples where there is not the sufficienct sample count
    Est_group = Est_group.loc[Est_group['count'] >= minimum_sample_count]
    
    # Create a list of point code names which meet the criteria
    Est_group_unique_with_dets = Est_group['ptcode'].unique()
    
    # Grab the coordinates from the est_group_all for later plotting
    Filtered_point_code_coords = {code: {'Easting': easting, 'Northing': northing}
              for code, easting, northing in zip(Est_group_all['ptcode'], Est_group_all['easting'], Est_group_all['northing'])}

    return Est_group_unique_with_dets, Filtered_point_code_coords


In [7]:
def monte_carlo_simulation_from_specific_years(early_period, late_period, samples, XX, num_iterations):

    trend_differences = []
    
    selected_early_data_indices = np.where((XX[:, 2] >= early_period[0]) & (XX[:, 2] <= early_period[1]))[0]
    selected_late_data_indices = np.where((XX[:, 2] >= late_period[0]) & (XX[:, 2] <= late_period[1]))[0]

    # Check if there are any samples available for the selected years
    if len(selected_early_data_indices) == 0 or len(selected_late_data_indices) == 0:
        print("No samples available for the selected years.")
        return None

    for x in range(num_iterations):

        idx1 = np.random.choice(selected_early_data_indices)
        idx2 = np.random.choice(selected_late_data_indices)
        
        sample_indices = random.sample(range(len(samples)), 2)
        selected_early_samples = samples[sample_indices[0],idx1]
        selected_late_samples = samples[sample_indices[1],idx2]

        # Calculate difference
        difference = (selected_late_samples - selected_early_samples)/(XX[idx2, 2] - XX[idx1,2])
        # Append to a dataframe
        trend_differences.append(difference)

    return trend_differences      

In [8]:
# Function to allow monte carlo simulations
def perform_monte_carlo_simulation(samples, XX_years, num_iterations, year_index):

    if XX_years[:,2].max() - XX_years[:,2].min() <= year_index:
        print(f"The range of years is not sufficient for analysis (less than or equal to {year_index} years).")
        return None
        
    trend_differences = []

    #Run through iteration
    for x in range(num_iterations):
        # Maximum number of attempts to find an appropriate pair of time segements to sample from
        max_attempts = 1000
        # Take a couple of random indices from the yearly dependence to sample from the normal distribution
        sample_indices = random.sample(range(len(samples)), 2)
        # Attempt to find a pair of numbers that are greater than 10 years apart
        for _ in range(max_attempts):
            # Allow the first yearly indices to be any number from 0 to n-1 (to allow for idx2)
            idx1 = np.random.randint(0, len(XX_years[:,2])-1)
            # Ensure idx2 is greater than idx1 (so the yearly change is in the right direction!)
            idx2 = np.random.randint(idx1 + 1, len(XX_years[:,2]))  # Ensuring idx2 is greater than idx1
            # Check the difference in years exceeds the year index
            if XX_years[:,2][idx2] - XX_years[:,2][idx1] >= year_index:
            #Take the random sample indices and the yearly indices for early and late samples
                sample_early = samples[sample_indices[0], idx1]
                sample_late = samples[sample_indices[1], idx2]
                # Calculate difference
                difference = (sample_late - sample_early)/(XX_years[idx2, 2] - XX_years[idx1,2])
                # Append to a dataframe
                trend_differences.append(difference)
                # Break the loop once the conditions are met
                break

    return trend_differences

In [9]:
#Note, there are numerous terms in this which need changing if the set of independent variables are altered.

def run_GAM(Est_group_unique_with_dets, Est_group_all, det, GAM_output_folder_path):
    
    #Define the pointcode list
    pointcode_list = Est_group_unique_with_dets

    #Set blank DF
    trend_result_list = []

    for pointcode in pointcode_list:
        # Assuming 'Est_group_all' is your original DataFrame
        Test_pt = Est_group_all.loc[Est_group_all['ptcode'] == pointcode].copy()  # Create a copy to avoid SettingWithCopyWarning

        # Date tp datetime
        Test_pt.loc[:, 'date'] = pd.to_datetime(Test_pt['date'])
        # Convert date to day of year
        Test_pt.loc[:, 'day_of_year'] = Test_pt['date'].dt.dayofyear

        # Apply the float_date function from above
        Test_pt['float_date'] = Test_pt['date'].apply(date_to_float_year)

        # This section converts time to a continuous scale
        # Extract hours and minutes
        hour = Test_pt['time'] // 100
        minutes = Test_pt['time'] % 100
        # Convert to continuous numeric format (minutes since midnight)
        Test_pt.loc[:, 'time_converted'] = hour * 60 + minutes

        # Handle NaN values if 'time' column contains them -CHECK THIS
        Test_pt['time_converted'] = Test_pt['time_converted'].fillna(-1)  # Replace NaN with a suitable value

        #Create independent variables
        X = Test_pt[['time_converted', 'day_of_year','float_date']].values
        #Create dependent variables
        Y = Test_pt[det].values

        # Remove rows with NaN values in Y. Note X gets removed in the same way.
        mask = ~np.isnan(Y)
        X = X[mask]
        Y = Y[mask]

        #Failsafe to break loop in case there is only one year in the data
        if len(np.unique(X[:, 2])) <= 1:
            print(f"Skipping analysis for pointcode {pointcode} as there is only one unique year.")
            continue  # Skip to the next iteration of the loop

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
        
        # Create and train the GAM model
        spline_no=5
        # Create and train the GAM model
        gam = GAM(s(0, n_splines=spline_no) + s(1,n_splines=spline_no,basis='cp') + s(2,n_splines=spline_no)).gridsearch(X_train, y_train)

        # Make predictions on the test set
        y_pred = gam.predict(X_test)

        # Evaluate the model
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        #======================PLOTTING BEGINS============================

        # Plotting actual vs predicted values
        plt.scatter(y_test, y_pred, alpha=0.5)
        # Sticking a 1:1 line in
        plt.plot([min(y_test)-1, max(y_test)+1], [min(y_test)-1, max(y_test)+1], '--', color='red', label='1:1 Line')
        plt.title(f'Actual vs Predicted Values - {pointcode}')
        plt.xlabel(f'Actual {det}')
        plt.ylabel(f'Predicted {det}')
        #Exporting figure
        Actual_predicted_path = os.path.join(GAM_output_folder_path, f'ActualVSPredicted_{pointcode}_{estuary}.png')
        plt.savefig(Actual_predicted_path )
        plt.clf()

        # Plotting residuals
        residuals = y_test - y_pred
        plt.scatter(y_pred, residuals, alpha=0.5)
        plt.title(f'Residuals Plot - {pointcode}')
        plt.xlabel(f'Predicted {det}')
        plt.ylabel('Residuals')
        plt.axhline(y=0, color='r', linestyle='--', linewidth=2)
        #Exporting figure
        Residuals_path = os.path.join(GAM_output_folder_path, f'Residuals_{pointcode}_{estuary}.png')
        plt.savefig(Residuals_path)
        plt.clf()

        #Plot the partial dependencies
        feature_names = ['time', 'day_of_year','float_date']

        fig, axs = plt.subplots(1, len(feature_names), figsize=(16, 4))
        for i, ax in enumerate(axs):
            XX = gam.generate_X_grid(term=i)
            pdep, confi = gam.partial_dependence(term=i, X=XX, width=0.95)
            ax.plot(XX[:, i], pdep)
            ax.plot(XX[:, i], confi, c='r', ls='--')
            ax.set_title(f'Partial dependence of {feature_names[i]}', fontsize = 10)
        plt.tight_layout()
        #Exporting figure
        Dependencies_path = os.path.join(GAM_output_folder_path, f'Dependencies_{pointcode}_{estuary}.png')
        plt.savefig(Dependencies_path)
        plt.clf()

        # This line extracts the partial dependence for the 'year' feature. There's probably a better way to combine with above code.
        XX = gam.generate_X_grid(term=2)
        pdep, confi = gam.partial_dependence(term=2, X=XX, width=0.95)

        # Plot the partial dependence for 'year'
        plt.plot(XX[:, 2], pdep)
        plt.fill_between(XX[:, 2], confi[:, 0], confi[:, 1], alpha=0.2, color='r')
        plt.title('Partial Dependence Plot for Year')
        plt.xlabel('Year')
        plt.ylabel('Partial Dependence')
        #Exporting figure
        Yearly_dependencies_path = os.path.join(GAM_output_folder_path, f'Yearly_dependencies_{pointcode}_{estuary}.png')
        plt.savefig(Yearly_dependencies_path)
        plt.close('all')
        
        #=======================
        
        # Take random samples from the dataset (X,Y) to show the uncertainty on the predictions
        for response in gam.sample(X, Y, quantity='y', n_draws=100, sample_at_X=XX):
            plt.scatter(XX[:, 2], response, alpha=0.03, color='k')  # Assuming XX[:, 0] is the appropriate feature for X-axis
        #print(response)
        plt.plot(XX[:, 2], gam.predict(XX), 'r--')  # Assuming XX[:, 0] is the appropriate feature for X-axis
        plt.title('Draw samples from the posterior of the coefficients')
        plt.xlabel('Year')
        plt.ylabel('Predicted temperature')
        Random_model_sample_dependencies_path = os.path.join(GAM_output_folder_path, f'Random_model_posterior_sample_distribution_{pointcode}_{estuary}.png')
        plt.savefig(Random_model_sample_dependencies_path)
        plt.close('all')

        # Extracting mean and confidence intervals from pdep and confi
        mean_year_dependence = pdep
        lower_bound = confi[:, 0]
        upper_bound = confi[:, 1]

        # Calculating standard deviation based on confidence intervals
        std_dev = (upper_bound - lower_bound) / (2 * 1.96)  # Assuming 1.96 corresponds to 95% confidence interval

        # Generating samples based on the partial dependence of 'year'
        num_samples = 100  # The number of samples you want to generate
        # Randomly sample assuming a normal distribution of variance
        samples = np.random.normal(mean_year_dependence, std_dev, size=(num_samples, len(mean_year_dependence)))

        # Plotting samples
        for sample in samples:
            plt.plot(XX[:, 2], sample, alpha=0.03, color='k')
        plt.plot(XX[:, 2], pdep, 'r--')  # Assuming XX[:, 3] is the appropriate feature for X-axis
        plt.title('Samples from the posterior of the coefficients')
        plt.xlabel('Year')
        plt.ylabel('Response')
        Random_yearly_dependency_sample_path = os.path.join(GAM_output_folder_path, f'Random_yearly_dependence_posterior_sample_distribution_{pointcode}_{estuary}.png')
        plt.savefig(Random_yearly_dependency_sample_path)
        plt.close('all')

        # Define the total number of iterations to run through monte carlo
        num_iterations = 10000

        # Set up blank results table
        trend_differences_year = []

        # =======================================================
        # HERE IS THE OPTIONS FOR HOW TO RUN THE TREND ANALYSIS - Comment one out
        # ======================================================

        # =/=/=/=/=/=/=/=/=/=/=/=/=/=//=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=
        # OPTION 1 = Specify two yearly periods

        #early_period = [2005.5,2006.5]
        #later_period=[2021.5,2022.5]
    
        #trend_differences_year = monte_carlo_simulation_from_specific_years(early_period, later_period, samples, XX, num_iterations)
        # =/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=
    
        # =/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=
        # OPTION 2 = Specifya yearly interval
    
        year_index = 10

        trend_differences_year = perform_monte_carlo_simulation(samples, XX, num_iterations, year_index)
        # =/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=/=

        # If trend_differences_year is None, continue to the next iteration
        if trend_differences_year is None:
            print(f"Skipping analysis for pointcode {pointcode} as monte_carlo_simulation_from_specific_years returned None.")
            continue

        # Plot histogram of trend differences to check the monte carlo simulation
        plt.hist(trend_differences_year, bins=50, color='red', edgecolor='black')
        plt.title('Histogram of Trend Differences')
        plt.xlabel('Difference in Trend')
        plt.ylabel('Frequency')
        plt.grid(True)
        Monte_carlo_histo_sample_path = os.path.join(GAM_output_folder_path, f'Monte_carlo_histogram_{pointcode}_{estuary}.png')
        plt.savefig(Monte_carlo_histo_sample_path)
        plt.close('all')
    
        # Calculate statistics of trend differences
        mean_difference = np.mean(trend_differences_year)
        std_dev_difference = np.std(trend_differences_year)
    
        # Print or use these statistics as needed
        print("Mean difference in trend:", mean_difference)
        print("Standard deviation of difference in trend:", std_dev_difference)

        #=============================Print to CSV============================================
        #Appending to a dictionaries for a csv
        trend_result_list.append({
            'pointcode': pointcode,
            'Average': mean_difference,
            'Std': std_dev_difference,
            'r_squared': r2,
            'Mean Squared Error': mse
        })
        # Convert the list of dictionaries to a DataFrame
        df = pd.DataFrame(trend_result_list)
        # Export the DataFrame to a CSV file
        csv_path = os.path.join(GAM_output_folder_path, f'Model_4_output_{estuary}_trend_result_list.csv')
        df.to_csv(csv_path, index=False)

        #=============================Print to Dict============================================
        # Append the information to the corresponding dictionary in Filtered_point_code_coords
        Filtered_point_code_coords[pointcode]['Average'] = mean_difference
        Filtered_point_code_coords[pointcode]['Std'] = std_dev_difference
        Filtered_point_code_coords[pointcode]['r_squared'] = r2
        Filtered_point_code_coords[pointcode]['Mean Squared Error'] = mse

        print(f'{pointcode} completed successfully')
    
    print('Run completed successfully')

    return trend_result_list, Filtered_point_code_coords

In [10]:
#Function to extract the estuary from the wider WFD TraC and coastal waterbody shapefile
def extract_shapefile(data_dict, estuary_to_extract, input_shapefile, target_crs='EPSG:xxxx'):
    if estuary_to_extract in data_dict:
        # Get the corresponding values from the dictionary
        corresponding_values = data_dict[estuary_to_extract]

        # Initialize an empty GeoDataFrame
        combined_gdf = gpd.GeoDataFrame()
    
        # Iterate through each corresponding value
        for value in corresponding_values:
            # Filter rows based on the current value
            subset_gdf = input_shapefile[input_shapefile['wb_name'] == value]

            # Check if the subset is not empty before combining
            if not subset_gdf.empty:
                # Concatenate the subset to the combined GeoDataFrame
                combined_gdf = pd.concat([combined_gdf, subset_gdf], ignore_index=True)

        # Check if there is more than one row for the key
        if len(combined_gdf) > 1:
            # Use unary_union to combine geometries
            combined_geometry = unary_union(combined_gdf['geometry'])

            # Create a new GeoDataFrame with the combined geometry
            new_gdf = gpd.GeoDataFrame(geometry=[combined_geometry], crs=target_crs)

            # Print or do further processing with the new GeoDataFrame
            print(new_gdf)
        elif not combined_gdf.empty:
            # Only one row, no need to combine, use the original GeoDataFrame
            new_gdf = combined_gdf.copy()
            new_gdf.crs = target_crs  # Set CRS for the single-row GeoDataFrame
            print(new_gdf)
        else:
            print(f"No features found for {estuary_to_extract}.")
    else:
        print(f"{estuary_to_extract} not found in the dictionary.")

    return new_gdf

In [11]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
print(parent_directory)
input_model_files_directory = os.path.join(parent_directory, "input_model_files")

C:\Users\alechutchings\GAM_models_temp_do


In [12]:
shapefile_directory = os.path.join(parent_directory, "wfd_shapefile")
wfd_trac_file_path = os.path.join(shapefile_directory, "WFD_Transitional_and_Coastal_Water_Bodies_Cycle_2.shp")
wfd_trac_shp = gpd.read_file(wfd_trac_file_path) 

In [13]:
csv_file_path = os.path.join(input_model_files_directory, "estuary_shapefile_dictionary.csv")
#csv_file_path = "C:\\Users\\alechutchings\\Documents\\PythonNotebooks\\estuary_shapefile_dictionary.csv"

# Initialize an empty dictionary
data_dict = {}

# Read the CSV file and populate the dictionary
with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    
    # Skip the header row if it exists
    next(csv_reader, None)
    
    # Iterate through rows and add key-value pairs to the dictionary
    for row in csv_reader:
        key = row[0]
        values = row[1:]  # Collect all values from the second column onwards
        data_dict[key] = values

In [14]:
# Create the estuary list
estuary_list = list(data_dict.keys())
print(len(estuary_list))

83


In [15]:
estuary_list = ['Poole']

#det = 'Oxygen, Dissolved as O2'
det = 'Temperature of Water'
#det = 'Oxygen, Dissolved,  Saturation'

#trend_sample_option = '1990_2022'
#trend_sample_option = '1990_2006'
#trend_sample_option = '2006_2022'
trend_sample_option = 'Yearly_10_interval'

master_df = pd.DataFrame()

for estuary in estuary_list:
    try:
        output_data_directory = os.path.join(parent_directory, "output_data")
        estuary_folder_path = os.path.join(output_data_directory,f'{estuary}')

        if not os.path.exists(estuary_folder_path):
            os.mkdir(estuary_folder_path)
            print(f"Folder '{estuary_folder_path}' created.")
        else:
            print(f"Folder '{estuary_folder_path}' already exists.")
            
        # Create an output plot folder for the GAM models
        folder_name = f'MODEL_4_GAM_outputs_individual_points_{det}_{trend_sample_option}'
        # Specify the full folder output
        GAM_output_path = os.path.join(estuary_folder_path, folder_name)

        # Set up a folder if it does not already exist
        if not os.path.exists(GAM_output_path):
            os.mkdir(GAM_output_path)
            print(f"Folder '{GAM_output_path}' created.")
        else:
            print(f"Folder '{GAM_output_path}' already exists.")

        # Specify the path for the csv improt
        csv_input_path = os.path.join(estuary_folder_path, f'{estuary}_wims_data_clean_nd_removed.csv')
        # Import the csv file
        df = import_csv(1,csv_input_path)
        #This is to ensure any pointcodes are kept constant (some have duplicates with blank spaces)
        df['ptcode'] = df['ptcode'].astype(str).str.strip()
        # Set up the pivot table
        Est_group_all = create_pivot(df)
        # Set up the filtering and pointcode list
        Est_group_unique_with_dets, Filtered_point_code_coords = filter_by_det(Est_group_all,det)
        # Run the GAM model code
        trend_result_list, Filtered_point_code_coords = run_GAM(Est_group_unique_with_dets, Est_group_all, det, GAM_output_path)

        # Extract the estuary shapefile
        est_shape = extract_shapefile(data_dict, estuary, wfd_trac_shp, target_crs='EPSG:27700')

        # Set up the geodataframe from the pointcodes specified in the GAM model
        gdf = gpd.GeoDataFrame(
            data=[{
                'pointcode': key,
                'Easting': value['Easting'],
                'Northing': value['Northing'],
                'geometry': Point(value['Easting'], value['Northing']),
                **value  # Add additional properties like Average, Std, etc.
            } for key, value in Filtered_point_code_coords.items()],
            geometry='geometry'  # Specify the geometry column
        )
        
        # Set the results list from the gam model to a dataframe 
        trend_result_list_df = pd.DataFrame(trend_result_list)
    
        # Calculate average (mean)
        average_value = trend_result_list_df['Average'].mean()

        # Calculate standard deviation
        std_deviation_value = trend_result_list_df['Average'].std()

        # Print or use the calculated values
        print(f'Average: {average_value}')
        print(f'Standard Deviation: {std_deviation_value}')

        # ============================ PLOTTING BEGINS FOR MAP ==============================================
        fig, ax0 = plt.subplots(nrows=1, ncols=1,figsize=(12, 10))
        # Plot the shapefile
        est_shape.plot(color='lightblue', ax=ax0)
        # Plot points with NaN values separately (if any)
        nan_color = 'gray'  # Choose a color for NaN values
        # Set up a cross if samples did not meet the criteria
        gdf[gdf['Average'].isna()].plot(ax=ax0, marker='x', markersize=75, edgecolor='grey', linewidth=0.5, legend_kwds={'label': "NaN"})

        # Plot the average values on the map
        gdf.plot(column='Average', cmap='coolwarm', vmin=-0.2, vmax=0.2, legend=True, ax=ax0, marker='o', markersize=75, edgecolor='black', linewidth=0.5, legend_kwds={'label': "Temperature increase per year derived from GAM (degrees celsius/yr)"})

        # Add a legend
        ax0.set_title(f'Distribution of trends across {estuary}')
        ax0.set_xlabel('Easting (m)')
        ax0.set_ylabel('Northing (m)')
        ax0.legend()
        map_path = os.path.join(GAM_output_path, f'{estuary}_GAM_map_{det}_{trend_sample_option}.png')
        plt.savefig(map_path)
        plt.close('all')
        # ====================== PLOTTING ENDS =================================================================
    
        # Add a column with the name of the estuary
        trend_result_list_df['Estuary'] = estuary
    
        # Append the DataFrame to a master DataFrame (if it already exists)
        if 'master_df' not in locals():
            master_df = trend_result_list_df.copy()
        else:
            master_df = pd.concat([master_df, trend_result_list_df], ignore_index=True)
    
    #Create an error log for issues but permit the loop to continue
    except Exception as e:
        # Print the error message
        error_message = f"Error processing {estuary}: {str(e)}"

        # Log the error to a notepad file
        log_file_path = os.path.join(output_data_directory,f'error_log_GAM_Model4_{det}_{trend_sample_option}.txt')
        with open(log_file_path, 'a') as log_file:
            log_file.write(error_message + '\n')
        print(error_message)

    finally:
        # Any cleanup code or additional actions you want to perform regardless of success or failure
        pass

# Now, master_df contains all the data with an additional 'Estuary' column. Save to csv for output
master_folder_path = os.path.join(output_data_directory,f"Model_4_output_file_{det}_{trend_sample_option}.csv")
master_df.to_csv(master_folder_path, index=False)



Folder 'C:\Users\alechutchings\GAM_models_temp_do\output_data\Poole' already exists.
Folder 'C:\Users\alechutchings\GAM_models_temp_do\output_data\Poole\MODEL_4_GAM_outputs_individual_points_Temperature of Water_Yearly_10_interval' created.


100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
 45% (5 of 11) |###########              | Elapsed Time: 0:00:00 ETA:   0:00:00

Mean difference in trend: 0.06153843170660032
Standard deviation of difference in trend: 0.01566425412169859
50590110 completed successfully


100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
 36% (4 of 11) |#########                | Elapsed Time: 0:00:00 ETA:   0:00:00

Mean difference in trend: 0.007436225870397081
Standard deviation of difference in trend: 0.05664249807468234
50900149 completed successfully


...

100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
 36% (4 of 11) |#########                | Elapsed Time: 0:00:00 ETA:   0:00:00

Mean difference in trend: 0.044436070140273225
Standard deviation of difference in trend: 0.05744211742712595
50950270 completed successfully


100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


Mean difference in trend: 0.07920357619776051
Standard deviation of difference in trend: 0.06359879911531768
50950300 completed successfully
Run completed successfully
            wb_id        wb_name  rbd_id    rbd_name        wb_cat  \
0  GB520804415800  POOLE HARBOUR     8.0  South West  Transitional   

     st_area_sh     st_perimet  \
0  3.309813e+07  157330.942598   

                                            geometry  
0  POLYGON ((404852.115 88281.046, 404839.000 882...  
Average: 0.03408886309358069
Standard Deviation: 0.02360200794313415


C:\Users\alechutchings\AppData\Local\anaconda3\envs\geo_env\Lib\site-packages\geopandas\plotting.py:313: UserWarning: You passed a edgecolor/edgecolors ('grey') for an unfilled marker ('x').  Matplotlib is ignoring the edgecolor in favor of the facecolor.  This behavior may change in the future.
  collection = ax.scatter(x, y, vmin=vmin, vmax=vmax, cmap=cmap, **kwargs)
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
